In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm

# DEMO

In [2]:
r = np.random.RandomState(seed=5)
ints = r.random_integers(1, 10, size=(3,2))
a = ['A', 'B', 'C']
b = ['D', 'A', 'E']
df = pd.DataFrame(ints, columns=['weight', 'cost'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  


In [3]:
df

,weight,cost
0,4,7
1,7,1
2,10,9


In [4]:
df[0] = a
df['b'] = b
df

,weight,cost,0,b
0,4,7,A,D
1,7,1,B,A
2,10,9,C,E


In [5]:
G=nx.from_pandas_edgelist(df, 0, 'b', ['weight', 'cost'])
G['E']['C']['weight']

10

In [6]:
G['E']['C']['cost']

9

# OUR DATASET

In [31]:
df=pd.read_csv("CAERS_ASCII_2004_2017Q2.csv",index_col="RA_Report #")

In [32]:
!ls

CAERS_ASCII_2004_2017Q2.csv  sample_data


In [33]:
df

,RA_CAERS Created Date,AEC_Event Start Date,PRI_Product Role,PRI_Reported Brand/Product Name,PRI_FDA Industry Code,PRI_FDA Industry Name,CI_Age at Adverse Event,CI_Age Unit,CI_Gender,AEC_One Row Outcomes,SYM_One Row Coded Symptoms
RA_Report #,,,,,,,,,,,
65325,1/1/2004,8/4/2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITAL..."
65325,1/1/2004,8/4/2003,Suspect,MIDWEST COUNTRY FAIR CHOCOLATE FLAVORED CHIPS,3,Bakery Prod/Dough/Mix/Icing,2.0,Year(s),Female,"VISITED AN ER, VISITED A HEALTH CARE PROVIDER,...","SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITAL..."
65333,1/1/2004,NaN,Suspect,KROGER CLASSIC CREAM-DE-MINT CANDY MINT CHIP I...,13,Ice Cream Prod,NaN,Not Available,Female,VISITED AN ER,"NAUSEA, DYSGEUSIA, DIARRHOEA"
65335,1/1/2004,11/24/2003,Suspect,ENFAMIL LIPIL BABY FORMULA,40,Baby Food Prod,3.0,Month(s),Not Available,NON-SERIOUS INJURIES/ ILLNESS,"GASTROINTESTINAL DISORDER, VOMITING"
65336,1/1/2004,NaN,Suspect,ENFIMIL LIPIL BABY FORMULA,40,Baby Food Prod,NaN,Not Available,Not Available,VISITED A HEALTH CARE PROVIDER,"GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION"
...,...,...,...,...,...,...,...,...,...,...,...
214516,6/29/2017,1/20/2017,Concomitant,CALTRATE CALCIUM,54,Vit/Min/Prot/Unconv Diet(Human/Animal),66.0,Year(s),Female,"HOSPITALIZATION, VISITED AN ER",SWELLING
214516,6/29/2017,1/20/2017,Concomitant,VITAMIN D3,54,Vit/Min/Prot/Unconv Diet(Human/Animal),66.0,Year(s),Female,"HOSPITALIZATION, VISITED AN ER",SWELLING
214516,6/29/2017,1/20/2017,Suspect,SUPERIOR PREFERENCE HAIR COLORING,53,Cosmetics,66.0,Year(s),Female,"HOSPITALIZATION, VISITED AN ER",SWELLING


## Now we have to select unique diseases and unique symptoms

In [34]:
df['SYM_One Row Coded Symptoms'].tolist()

['SWELLING FACE, RASH, WHEEZING, COUGH, HOSPITALISATION, DYSPNOEA',
 'SWELLING FACE, WHEEZING, COUGH, RASH, HOSPITALISATION, DYSPNOEA',
 'NAUSEA, DYSGEUSIA, DIARRHOEA',
 'GASTROINTESTINAL DISORDER, VOMITING',
 'GASTROINTESTINAL DISORDER, PHYSICAL EXAMINATION',
 'CHOKING',
 'DEATH, CHOKING',
 'PARANOIA, PHYSICAL EXAMINATION, DELUSION',
 'PARANOIA, PHYSICAL EXAMINATION, DELUSION',
 'BLOOD PRESSURE INCREASED',
 'RASH, FLUSHING, MALAISE, FAECES PALE, VOMITING, ABDOMINAL PAIN, DIARRHOEA, OROPHARYNGEAL SWELLING, PHYSICAL EXAMINATION, LOOSE STOOLS',
 'LOOSE STOOLS, ABDOMINAL PAIN UPPER, VOMITING, ABDOMINAL PAIN, RASH',
 'RASH, LOOSE STOOLS, ABDOMINAL PAIN UPPER, MALAISE, DIARRHOEA, ABDOMINAL PAIN, VOMITING',
 'SWELLING FACE, PARAESTHESIA ORAL',
 'PHARYNGITIS',
 'VOMITING',
 'COMPLETED SUICIDE, STRESS SYMPTOMS, DEATH',
 'DEATH, MITRAL VALVE INCOMPETENCE',
 'CEREBROVASCULAR ACCIDENT',
 'MUCOSAL DISCOLOURATION, SKIN DISCOLOURATION',
 'HEART RATE INCREASED, DIZZINESS, BLOOD PRESSURE INCREASED, PA

In [35]:
#There are NaNs in symptoms, we need to rename them into None
df["SYM_One Row Coded Symptoms"].fillna(value="No Symptoms",inplace=True)
#There are leading spaces... We need to omit them
df["SYM_One Row Coded Symptoms"]=df["SYM_One Row Coded Symptoms"].str.replace(', ', ',')
df["AEC_One Row Outcomes"]=df["AEC_One Row Outcomes"].str.replace(', ', ',')

In [36]:
for l in df["SYM_One Row Coded Symptoms"].tolist():
    print(l.split(","))

Streaming output truncated to the last 5000 lines.
['OVARIAN CANCER']
['OVARIAN CANCER']
['MALAISE', 'FATIGUE', 'DIZZINESS', 'VOMITING', 'PAIN', 'SYNCOPE', 'INFLUENZA', 'ASTHENIA']
['VOMITING', 'DIARRHOEA', 'TREMOR', 'HYPERVENTILATION', 'CHILLS', 'MUSCULOSKELETAL STIFFNESS', 'PYREXIA']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['CHEST DISCOMFORT', 'FATIGUE', 'DYSPNOEA', 'CARDIAC DISORDER']
['HEART RATE INCREASED', 'ASTHENIA', 'TREMOR']
['HAEMORRHAGE', 'GASTROINTESTINAL HAEMORRHAGE', 'PROTHROMBIN LEVEL ABNORMAL', 'HAEMORRHOIDS']
['BLOOD URINE PRESENT', 'HAEMORRHAGE', 'CONSTIPATION']
['BLO

In [37]:
i=2
for l in df["AEC_One Row Outcomes"].tolist():
    try:
      l.split(",")
    except AttributeError:
      print(l,i)
    i=i+1

In [38]:
# I had to convert to string bcz some numbers are there -- It was actually missing values..
# Here I have already rectified it by replacing it with "No Symptoms" already
Symptoms = {x for l in df["SYM_One Row Coded Symptoms"].tolist() for x in str(l).split(",")}

In [39]:
Symptoms # I had to make it a set to remove repetitions

{'INCREASED INSULIN REQUIREMENT',
 'ABDOMINAL DISCOMFORT',
 'PATHOLOGY TEST',
 'DIFFICULTY IN WALKING',
 'VULVOVAGINAL DISCOMFORT',
 'DIALYSIS',
 'CERVIX NEOPLASM',
 'HYPERPARATHYROIDISM SECONDARY',
 'VITAMIN B1 DEFICIENCY',
 'HYPEREOSINOPHILIC SYNDROME',
 'FINGER DEFORMITY',
 'PELVIC MASS',
 'NECK DEFORMITY',
 'METASTASES TO PERITONEUM',
 'URTICARIA GENERALISED',
 'BRONCHIOLOALVEOLAR CARCINOMA',
 'PICA',
 'LOCAL SWELLING',
 'VAGINAL POLYP',
 'LIPOHYPERTROPHY',
 'RED CELL DISTRIBUTION WIDTH DECREASED',
 'BLOOD LACTATE DEHYDROGENASE INCREASED',
 'ERECTION INCREASED',
 'ARTERIAL STENT INSERTION',
 'EJACULATION DISORDER',
 'AZOTAEMIA',
 'INVASIVE LOBULAR BREAST CARCINOMA',
 'UROLOGICAL EXAMINATION',
 'AKATHISIA',
 'STRONGYLOIDIASIS',
 'NERVOUSNESS',
 'DEAFNESS',
 'SLOW VIRUS INFECTION',
 'SKIN DISCOLOURATION',
 'LIGAMENT RUPTURE',
 'ONYCHOLYSIS',
 'LIVER INJURY',
 'BONE MARROW TRANSPLANT',
 'BUNDLE BRANCH BLOCK RIGHT',
 'SKIN HYPOPIGMENTATION',
 'INSULIN TOLERANCE TEST NORMAL',
 'CHONDROS

In [40]:
Outcome = {x for l in df["AEC_One Row Outcomes"].tolist() for x in l.split(",")} # No need to convert to string

In [42]:
Outcome

{'CONGENITAL ANOMALY',
 'DEATH',
 'DISABILITY',
 'HOSPITALIZATION',
 'LIFE THREATENING',
 'NON-SERIOUS INJURIES/ ILLNESS',
 'NONE',
 'OTHER SERIOUS (IMPORTANT MEDICAL EVENTS)',
 'REQ. INTERVENTION TO PRVNT PERM. IMPRMNT.',
 'SERIOUS INJURIES/ ILLNESS',
 'VISITED A HEALTH CARE PROVIDER',
 'VISITED AN ER'}

In [43]:
from itertools import product
Graph_df=pd.DataFrame(list(product(Symptoms,Outcome)), columns=['Symptoms', 'Outcome'])

In [44]:
Graph_df['Weight']=0 #adding a zero column

In [45]:
Graph_df

,Symptoms,Outcome,Weight
0,INCREASED INSULIN REQUIREMENT,VISITED AN ER,0
1,INCREASED INSULIN REQUIREMENT,LIFE THREATENING,0
2,INCREASED INSULIN REQUIREMENT,NONE,0
3,INCREASED INSULIN REQUIREMENT,SERIOUS INJURIES/ ILLNESS,0
4,INCREASED INSULIN REQUIREMENT,REQ. INTERVENTION TO PRVNT PERM. IMPRMNT.,0
...,...,...,...
49783,TRIGGER FINGER,DEATH,0
49784,TRIGGER FINGER,NON-SERIOUS INJURIES/ ILLNESS,0
49785,TRIGGER FINGER,DISABILITY,0
49786,TRIGGER FINGER,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),0


# Now we have to see which Symptoms are connected to which outcome

We will traverse the given dataset and once we get connected symptom and outcome we update their weight by one

In [46]:
#This cell is a bottleneck... Adding logs
#Started at 10:49......
#Should include tqdm and should check for better methods
#Took almost 6 minutes but stuck in that float symptom... couldn't split... should convert that col to str after inspecting that float value
#Restarted at 10:56
#I think str conversion is another bottleneck--- at 11:06
#It is 11:55 now, still not finished... I am going to add tqdm now...
#Added tqdm at 12:02.. tqdm indicates it need almost 2hrs to finish... should have calculated the propotions too now itself!!!
#It is 12:47 and .. 1/3 of iterations is done...on 45~ minutes
for row in tqdm(range(0,len(df))):
    list1=df['AEC_One Row Outcomes'].iloc[row].split(",")
    list2=df['SYM_One Row Coded Symptoms'].iloc[row,].split(",")
    for item1 in list1:
        for item2 in list2:
            ind=Graph_df.loc[(Graph_df['Outcome'] == item1) & (Graph_df['Symptoms'] == item2)].index[0]
            Graph_df['Weight'].iloc[ind]+=1

  0%|          | 0/90786 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 90786/90786 [1:03:53<00:00, 23.68it/s]


In [47]:
Graph_df.loc[(Graph_df['Outcome'] == 'VISITED AN ER') & (Graph_df['Symptoms'] == 'RASH')]

,Symptoms,Outcome,Weight
44652,RASH,VISITED AN ER,666


In [48]:
Graph_df

,Symptoms,Outcome,Weight
0,INCREASED INSULIN REQUIREMENT,VISITED AN ER,0
1,INCREASED INSULIN REQUIREMENT,LIFE THREATENING,0
2,INCREASED INSULIN REQUIREMENT,NONE,0
3,INCREASED INSULIN REQUIREMENT,SERIOUS INJURIES/ ILLNESS,0
4,INCREASED INSULIN REQUIREMENT,REQ. INTERVENTION TO PRVNT PERM. IMPRMNT.,0
...,...,...,...
49783,TRIGGER FINGER,DEATH,0
49784,TRIGGER FINGER,NON-SERIOUS INJURIES/ ILLNESS,0
49785,TRIGGER FINGER,DISABILITY,0
49786,TRIGGER FINGER,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),1


In [49]:
Graph_df = Graph_df[Graph_df.Weight != 0]

In [50]:
Graph_df

,Symptoms,Outcome,Weight
10,INCREASED INSULIN REQUIREMENT,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),1
12,ABDOMINAL DISCOMFORT,VISITED AN ER,217
13,ABDOMINAL DISCOMFORT,LIFE THREATENING,67
14,ABDOMINAL DISCOMFORT,NONE,3
15,ABDOMINAL DISCOMFORT,SERIOUS INJURIES/ ILLNESS,72
...,...,...,...
49761,GALLBLADDER INJURY,DISABILITY,3
49762,GALLBLADDER INJURY,OTHER SERIOUS (IMPORTANT MEDICAL EVENTS),7
49763,GALLBLADDER INJURY,VISITED A HEALTH CARE PROVIDER,3
49765,EXAGGERATED STARTLE RESPONSE,LIFE THREATENING,5


In [51]:
G=nx.from_pandas_edgelist(Graph_df, 'Symptoms', 'Outcome', ['Weight', 'Weight'])
G['RASH']['VISITED AN ER']['Weight']

666

In [52]:
nx.write_gexf(G,"symptomAndOutcomes.gexf")

In [53]:
Graph_df.to_csv('Graph.csv', index=False)

fatal: not a git repository (or any of the parent directories): .git
